In [ ]:
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Dropout
from keras.callbacks import EarlyStopping
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from keras.models import load_model
from sklearn.model_selection import train_test_split
from skimage import data, color
from skimage.transform import rescale, resize, downscale_local_mean
from skimage import io
from skimage.color import rgba2rgb
from skimage.color import gray2rgb
import numpy as np
import pandas as pd
import os
from google.colab import drive
import tensorflow as tf
from PIL import Image

Zajmujemy się przewidywaniem na podstawie zdjęć rodzaju wnętrza(kuchnia, łazienka itd.), znajdujących się na nich obiektów(krzesło wannna...) oraz oceny stanu nieruchomości.

In [ ]:
def f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return 1-f1_val


In [ ]:
def focal_loss(y_true, y_pred):
    gamma = 2.0
    alpha = 0.25
    pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
    pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))
    return -K.sum(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1))-K.sum((1-alpha) * K.pow( pt_0, gamma) * K.log(1. - pt_0))

In [ ]:
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
base_model_2 = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(224,224,3))

# add a global spatial average pooling layer
x2 = base_model_2.output
x2 = GlobalAveragePooling2D()(x2)
# let's add a fully-connected layer
x2 = Dense(1024, activation='relu')(x2)
state = Dense(4, activation='softmax')(x2)
condition = Dense(4, activation='softmax')(x2)

# this is the model we will train
model_2 = Model(inputs=base_model_2.input, outputs=[state, condition])














219062272/219055592 [==============================] - 6s 0us/step


In [ ]:
base_model = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(224,224,3))

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
stuff = Dense(53, activation='sigmoid')(x)
room = Dense(6, activation='softmax')(x)

# # this is the model we will train
model = Model(inputs=base_model.input, outputs=[stuff, room])

In [ ]:
for layer in base_model.layers:
    layer.trainable = False
    

In [ ]:
labels_task_1 = ['Bathroom', 'Bathroom cabinet', 'Bathroom sink', 'Bathtub', 'Bed', 'Bed frame',
                 'Bed sheet', 'Bedroom', 'Cabinetry', 'Ceiling', 'Chair', 'Chandelier', 'Chest of drawers',
                 'Coffee table', 'Couch', 'Countertop', 'Cupboard', 'Curtain', 'Dining room', 'Door', 'Drawer',
                 'Facade', 'Fireplace', 'Floor', 'Furniture', 'Grass', 'Hardwood', 'House', 'Kitchen',
                 'Kitchen & dining room table', 'Kitchen stove', 'Living room', 'Mattress', 'Nightstand',
                 'Plumbing fixture', 'Property', 'Real estate', 'Refrigerator', 'Roof', 'Room', 'Rural area',
                 'Shower', 'Sink', 'Sky', 'Table', 'Tablecloth', 'Tap', 'Tile', 'Toilet', 'Tree', 'Urban area',
                 'Wall', 'Window']

labels_task2 = ['bathroom', 'bedroom', 'dinning_room', 'house', 'kitchen', 'living_room']

labels_task3 = [1,2,3,4]

root = os.path.join(os.getcwd(),"gdrive","My Drive","skyhacks")
rootdir = os.path.join(root, "resizeduni/")

#imgcsv = pd.read_csv(os.path.join(root,"labels.csv")) #csv with image info
imgcsv = pd.read_csv("labels.csv")
imgcsv2 = imgcsv[imgcsv["task2_class"]!="validation"] #csv with image info without images with "validation" room class

targets = imgcsv2.drop(["filename", "standard","task2_class", "tech_cond"], axis = 1).to_numpy()
targets2 = pd.get_dummies(imgcsv2, columns = ["task2_class"])[["task2_class_bathroom","task2_class_bedroom",
                                                               "task2_class_dining_room","task2_class_house","task2_class_kitchen","task2_class_living_room"]].to_numpy()
targets3 = to_categorical(imgcsv.drop(["filename", "task2_class", "tech_cond"]+labels_task_1, axis = 1).to_numpy()-1)
targets4 = to_categorical(imgcsv.drop(["filename", "task2_class", "standard"]+labels_task_1, axis = 1).to_numpy()-1)



In [ ]:
os.chdir(os.path.join(root, "resizeduni/"))

FileNotFoundError: ignored

In [ ]:
os.listdir()

['proc',
 'run',
 'root',
 'lib',
 'lib64',
 'bin',
 'sys',
 'usr',
 'sbin',
 'media',
 'boot',
 'dev',
 'home',
 'var',
 'tmp',
 'mnt',
 'srv',
 'opt',
 'etc',
 'content',
 '.dockerenv',
 'tensorflow-2.0.0',
 'tools',
 'datalab',
 'swift',
 'lib32']

In [ ]:
imgset = []

for name in imgcsv2["filename"]:
    imgset.append(io.imread(os.path.join(rootdir, name))/255)
    
dataset = np.array([np.array(imgset[i]) for i in range(len(imgset))])    

FileNotFoundError: ignored

In [ ]:
imgset_2 = []

for name in imgcsv["filename"]:
    imgset_2.append(io.imread(os.path.join(rootdir, name))/255)
    
dataset_2 = np.array([np.array(imgset_2[i]) for i in range(len(imgset_2))])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(dataset, targets, test_size=0.2, random_state = 42)
X_train, X_test, y_train2, y_test2 = train_test_split(dataset, targets2, test_size = 0.2, random_state = 42)

EarlyStop = EarlyStopping(monitor= "val_dense_6_f1", patience = 5, restore_best_weights=True)

losses = {
	"dense_5": "binary_crossentropy",
	"dense_6": "categorical_crossentropy",
}

model.compile(optimizer='adam', loss=losses, metrics = [f1])

model.fit(X_train,
 	{"dense_5": y_train, "dense_6": y_train2},
	validation_data=(X_test,
		{"dense_5": y_test, "dense_6": y_test2}), epochs=50, callbacks = [EarlyStop], batch_size = 16)

In [ ]:
X_train_2, X_test_2, y_train3, y_test3 = train_test_split(dataset_2, targets3, test_size=0.2, random_state = 42)
X_train_2, X_test_2, y_train4, y_test4 = train_test_split(dataset_2, targets4, test_size = 0.2, random_state = 42)

class_weight = {
    1: 1,
    2: 1,
    3: 10,
    4: 6
}

EarlyStop2 = EarlyStopping(monitor= "val_dense_3_f1", patience = 5, restore_best_weights=True)

losses_2 = {
	"dense_2": focal_loss,
	"dense_3": "categorical_crossentropy"
}

model_2.compile(optimizer='adam', loss=losses_2, metrics = [f1])

model_2.fit(X_train_2,
	{"dense_2": y_train3, "dense_3": y_train4},
	validation_data=(X_test_2,
		{"dense_2": y_test3, "dense_3": y_test4}), epochs=50, callbacks = [EarlyStop2], 
            batch_size=16, class_weight=class_weight)

In [ ]:
testset = []
ans = np.zeros((30, 57), dtype="object")
rootdir = os.path.join(root, "resized_test_dataset/")
for i, name in enumerate(os.listdir(rootdir)):
    ans[i][0]=name
    testset.append(io.imread(os.path.join(rootdir, name))/255)
    
testset = np.array([np.array(testset[i]) for i in range(len(testset))])

In [ ]:
y = model.predict(testset)

In [ ]:
for i in range(len(y[0])):
    for j in range(len(y[0][0])):
        ans[i][j+4]= 1 if y[0][i][j] >=0.5 else 0

In [ ]:
for i in range(len(y[1])):
  ans[i][2] = labels_task2[np.argmax(y[1][i])]

In [ ]:
y2 = model_2.predict(testset)

In [ ]:
for i in range(len(y2[0])):
  ans[i][1] = labels_task3[np.argmax(y2[0][i])]
  ans[i][3] = labels_task3[np.argmax(y2[1][i])]
 

In [ ]:
df = pd.DataFrame(ans,columns=['filename', 'standard', 'task2_class', 'tech_cond'] + labels_task_1)
df.to_csv("ans.csv", index = False)#,line_terminator="n")

In [ ]:
folder = "test"

for file in os.listdir(folder):
  im = Image.open(os.path.join(folder,file))
  im.convert("RGB")
  im.resize((224,224))
  im.save(file)
  im.close